In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 라이브러리 불러오기

In [2]:
import networkx as nx
import operator

In [4]:
import Umorpheme.morpheme as um
from collections import OrderedDict

s = '형태소 분석기 유몰핌입니다'
server = 'http://information.center/api/korean'
apikey = 'koorJRK3XO1G0W' # Register at http://information.center/korean
data = um.analyzer(s, server, apikey, '유몰핌,유니스트', 1)

temp = {}
for key, value in data.items():
    temp[int(key)] = value
data = OrderedDict(sorted(temp.items()))

for i, j in data.iteritems():
    print i, j['data'], j['feature']

0 형태소 분석기 COMP
2 유몰핌 CUSTOM
3 입니다 VCP+EC


### 명사 추출 함수

In [5]:
def nouns(s):
    """ 명사 추출 함수 """
    if type(s) == unicode:
        s = s.encode('utf8')
    server = 'http://information.center/api/korean'
    apikey = 'koorJRK3XO1G0W' # Register at http://information.center/korean
    data = um.analyzer(s, server, apikey, '유몰핌,유니스트', 1)

    temp = {}
    for key, value in data.items():
        temp[int(key)] = value
    data = OrderedDict(sorted(temp.items()))

    selected = []
    for i, j in data.iteritems():
        if j['feature'] in ['COMP', 'NNP', 'NNG']:
            selected.append(j['data'])
    return selected

### 텍스트 파일 불러와서 news 리스트로 만들기

In [6]:
filename = 'CHOSUN_2000.txt'
f = open(filename, 'r')
data = f.read()
f.close()
data = data.decode('cp949')
newslist = data.split('</DOC>') # 뉴스 리스트

news = list()
for n in newslist:
    if '<TEXT>' in n:
        content = n.split('<TEXT>')[1].split('</TEXT>')[0]
        news.append(content)

# 뉴스 하나만 네트워크로

### news[0] 에서 명사만 keys로 저장

In [7]:
keys = nouns(news[0])

In [8]:
# 키 쌍 보기
for idx, key in enumerate(keys[:-1]):
    print keys[idx], '/', keys[idx+1]

시작 / 서울 광화문 이순신 장군 동상 앞
서울 광화문 이순신 장군 동상 앞 / 설치
설치 / 대형 시계추 숫자
대형 시계추 숫자 / 광화문 일대
광화문 일대 / 폭죽
폭죽 / 대형 카드 섹션
대형 카드 섹션 / 자
자 / 김창종 기자


#### news[0]의 keys들을 네트워크로 만들기

In [9]:
G = nx.Graph()
for idx, key in enumerate(keys[:-1]):
    G.add_edge(keys[idx], keys[idx+1])

# 뉴스 여러 개를 네트워크로

### news[:10], 다시 말해 0부터 10개의 뉴스에 대해서 네트워크 만들기

In [13]:
G = nx.Graph()
for num, n in enumerate(news[:10]):
    keys = nouns(n)
    for idx, key in enumerate(keys[:-1]):
        G.add_edge(keys[idx], keys[idx+1])
    print num

0
1
2
3
4
5
6
7
8
9


##### 네트워크 정보 보기

In [14]:
print "Number of Nodes : ", nx.number_of_nodes(G)
print "Number of Edges : ", nx.number_of_edges(G)
degreelist = list(G.degree().values())
print "Avg. Node Degree : ", float(sum(degreelist))/nx.number_of_nodes(G)
try:
    print "Avg. Path Length : ", nx.average_shortest_path_length(G)
except nx.NetworkXError as e:
    print e
print "Avg. Clustering Coefficient : ", nx.average_clustering(G)

Number of Nodes :  641
Number of Edges :  802
Avg. Node Degree :  2.5023400936
Avg. Path Length :  9.98178139626
Avg. Clustering Coefficient :  0.0238865268896


#### 매개 중심성 계산

In [15]:
print "::: Betweenness Centrality"
x = nx.betweenness_centrality(G)
sorted_list = sorted(x.iteritems(), key=operator.itemgetter(1), reverse=True)
for s in sorted_list[:20]:
    print s[0], s[1]

::: Betweenness Centrality
민족 0.222541767307
확대 0.207124594332
계획 0.184870070443
인터넷 0.18424020139
관련 기사 0.164401952432
장애 0.146719830063
말 0.136220016073
정신 0.136107409275
창당 0.109373946427
보수 신 0.103069256792
오늘 0.0939545238793
때 0.0936252992899
지명 0.0920145977984
각종 수혜 0.0887227150783
선 0.0880045315084
조선일보 0.0853922813569
전국 0.0815975921005
환자 0.0808245548628
사실 0.0746019420557
일본 정부 0.0744875378165


#### 근접 중심성 계산

In [16]:
print "::: Closeness Centrality"
x = nx.closeness_centrality(G)
sorted_list = sorted(x.iteritems(), key=operator.itemgetter(1), reverse=True)
for s in sorted_list[:20]:
    print s[0], s[1]

::: Closeness Centrality
민족 0.149812734082
정신 0.14398200225
확대 0.143820224719
때 0.14343343792
계획 0.142697881828
장애 0.142159040426
선 0.14053579271
국가 0.140043763676
경우 등록 장애 0.138228941685
관련 기사 0.137990513152
전문가 0.137486573577
급 0.13736853402
환자 0.136927685066
각종 수혜 0.136431464507
전망 0.135765803988
혼 0.135507092949
본 0.135421074905
르네상스 운동 0.134708482425
제정신 0.134651798864
한반도 0.134623474968


# 그래프 데이터 추출하기 for Gephi

In [45]:
nx.write_gexf(G, 'graph.gexf')